In [3]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from catboost import CatBoostClassifier, Pool

# Настройки отображения
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 30)
pd.options.display.float_format = '{:.2f}'.format

# Инициализация путей
root = Path(os.getcwd()).parent.parent
data_in = root / 'data' / 'eda_data'
data_out = root / 'data' / 'model_data'

print("Пути инициализированы:")
print(f"EDA data: {data_in}")
print(f"Model data: {data_out}")

Пути инициализированы:
EDA data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/eda_data
Model data: /Users/aleksey.sushchikh/Desktop/GitHub/MIFIHackatonSberAutoSubscriptionAnalysis/data/model_data


In [5]:
# Загрузка данных
print("\nЗагружаем предобработанные данные...")
data = pd.read_pickle(data_in / 'eda_data.pkl')
print("\nДанные успешно загружены!")


Загружаем предобработанные данные...

Данные успешно загружены!


In [6]:
 def generate_summary(df: pd.DataFrame, name: str, sample_size: int = 3) -> None:
    """Генерация расширенной сводки по данным примерами"""
    print(f"\n{'='*50} {name.upper()} {'='*50}")
    print(f"Общее количество записей: {df.shape[0]:,}")
    print(f"Количество признаков: {df.shape[1]}")
    
    # Типы данных
    print("\nТипы данных:")
    print(df.dtypes.value_counts().rename('count').to_frame())
    
    # Пропуски
    missing = df.isna().sum().sort_values(ascending=False)
    missing_pct = (missing / df.shape[0] * 100).round(2)
    missing_df = pd.concat([missing, missing_pct], axis=1, keys=['count', '%']).query('count > 0')
    if not missing_df.empty:
        print("\nПропущенные значения:")
        print(missing_df)
    else:
        print("\nПропущенных значений нет")
        
    # Дубликаты
    dupes = df.duplicated().sum()
    print(f"\nДубликаты: {dupes} ({dupes/df.shape[0]*100:.2f}%)")
    
    # Примеры
    print(f"\nПервые {sample_size} записей:")
    display(df.head(sample_size))


def analyze_column(df: pd.DataFrame, col: str, max_display: int = 50) -> None:
    """Полный анализ колонки с выводом всех уникальных значений"""
    print(f"\n{'-'*60}")
    print(f"Полный анализ колонки: {col}")
    
    # Проверка существования колонки
    if col not in df.columns:
        print(f"Колонка {col} не найдена!")
        return
    
    # Пропуски
    na_count = df[col].isna().sum()
    print(f"Пропуски: {na_count} ({na_count/len(df)*100:.1f}%)")

    # Количество уникальных значений
    unique_count = df[col].nunique(dropna=False)
    print(f"Уникальных значений: {unique_count}")
    
    # Вывод всех значений для категориальных данных
    if unique_count <= max_display:
        print("\nВсе значения:")
        print(df[col].unique())
    else:
        print(f"\nСлишком много значений (> {max_display}). Примеры:")
        print(df[col].dropna().sample(10).unique())
        
    # Частотный анализ для числовых колонок
    if pd.api.types.is_numeric_dtype(df[col]):
        print("\nОписательная статистика:")
        print(df[col].describe())
    else:
        print("\nТоп-10 значений:")
        print(df[col].value_counts(dropna=False).head(10))

In [7]:
generate_summary(data, "Данные")
for col in data.columns:
    analyze_column(data, col, max_display=100)


================================================== ДАННЫЕ ==================================================
Общее количество записей: 13,337,484
Количество признаков: 35

Типы данных:
         count
object      18
int64       14
float64      3

Пропущенных значений нет

Дубликаты: 0 (0.00%)

Первые 3 записей:


,session_id,client_id,visit_hour,visit_weekday,is_weekend,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_screen_height,device_screen_width,device_browser,geo_country,geo_city,hit_number,hit_referer,hit_page_path,event_category,event_action,event_label,is_target,aspect_ratio,n_hits,n_target_hits,n_unique_pages,visit_day,visit_month,visit_year,hit_sec,session_total_sec
0,9055434745589932991.163775...,2108382700.1637753791,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,other_russia_city,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0,0.50,4,1,1,24,11,2021,3.67,42.93
1,9055434745589932991.163775...,2108382700.1637753791,14,2,0,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,720,360,Chrome,russia,other_russia_city,4,missing,podpiska.sberauto.com/,sub_button_click,sub_view_cars_click,vodKSlUobUWTVlgsJqdI,1,0.50,4,1,1,24,11,2021,46.59,42.93
2,905544597018549464.1636867...,210838531.1636867288,8,6,1,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,854,385,Samsung Internet,russia,Moscow,3,missing,podpiska.sberauto.com/,sub_page_view,sub_landing,missing,0,0.45,3,0,1,14,11,2021,0.92,0.00



------------------------------------------------------------
Полный анализ колонки: session_id
Пропуски: 0 (0.0%)
Уникальных значений: 1582604

Слишком много значений (> 100). Примеры:
['3042541339328468409.1629312441.1629312441'
 '2894856024120230579.1637351240.1637351240'
 '1942761319051052300.1634223373.1634223373'
 '2702455576754299266.1639358852.1639358852'
 '4231418669469263159.1638655287.1638655287'
 '5190191846813268697.1634663128.1634663189'
 '6331865514976198580.1639169976.1639169976'
 '8018517636180057578.1639423683.1639423683'
 '4808640842163295807.1637094975.1637094975'
 '2991116158287741259.1621885258.1621885258']

Топ-10 значений:
session_id
1544572560279928739.1632436140.1632436140    120
5692861315757623740.1632356796.1632356796    120
6766315690481471549.1632423998.1632423998    116
6556325878862444653.1632368752.1632368752    116
6344040849605405862.1632382118.1632382118    116
4781656099508856169.1632511340.1632511340    116
477529334778357839.1632299092.1632299092

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

df = data.copy()

categorical_features = [
    'utm_source', 'utm_medium', 'utm_campaign', 'utm_adcontent', 'utm_keyword',
    'device_category', 'device_os', 'device_brand', 'device_browser',
    'geo_country', 'geo_city'
]

# Отдельный таргет
X = df.drop(['session_id', 'is_target'], axis=1)
y = df['is_target']

# Статическое разбиение
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

train_pool = Pool(
    data=X_train,
    label=y_train,
    cat_features=categorical_features
)
val_pool = Pool(
    data=X_val,
    label=y_val,
    cat_features=categorical_features
)

for col in X.columns:
    X_col = X[[col]].copy()
    
    if X_col[col].dtype == 'object':
        le = LabelEncoder()
        try:
            X_col[col] = le.fit_transform(X_col[col])
        except:
            print(f"Пропущен {col} — не удалось закодировать")
            continue
    
    model = DecisionTreeClassifier(max_depth=1)
    model.fit(X_col, y)
    score = model.score(X_col, y)
    print(f"{col}: {score:.4f}")

correlations = X.select_dtypes(include=['float', 'int']).corrwith(y).abs().sort_values(ascending=False)
print(correlations)

duplicates = pd.merge(X_train, X_val, how='inner')
print(f"Дубликатов между train и val: {len(duplicates)}")

client_id: 0.8387
visit_hour: 0.8387
visit_weekday: 0.8387
is_weekend: 0.8387
visit_number: 0.8387
utm_source: 0.8387
utm_medium: 0.8387
utm_campaign: 0.8387
utm_adcontent: 0.8387
utm_keyword: 0.8387
device_category: 0.8387
device_os: 0.8387
device_brand: 0.8387
device_screen_height: 0.8387
device_screen_width: 0.8387
device_browser: 0.8387
geo_country: 0.8387
geo_city: 0.8387
hit_number: 0.8387
hit_referer: 0.8387
hit_page_path: 0.8387
event_category: 0.8387
event_action: 0.8921
event_label: 0.8387
aspect_ratio: 0.8387
n_hits: 0.8387
n_target_hits: 0.8437
n_unique_pages: 0.8387
visit_day: 0.8387
visit_month: 0.8387
visit_year: 0.8387
hit_sec: 0.8387
session_total_sec: 0.8387


/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/opt/anaconda3/lib/python3.12/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


n_target_hits          0.32
aspect_ratio           0.05
device_screen_width    0.05
n_hits                 0.05
hit_number             0.05
device_screen_height   0.03
visit_month            0.03
visit_number           0.02
visit_day              0.02
n_unique_pages         0.01
is_weekend             0.01
visit_weekday          0.01
hit_sec                0.01
session_total_sec      0.00
visit_hour             0.00
visit_year              NaN
dtype: float64
Дубликатов между train и val: 0


"\n# X и y\nX = data.drop('is_target', axis=1)\ny = data['is_target']\n\n# создаём Pool (рекомендуется при наличии категориальных)\ncat_feats = ['session_id','client_id','utm_source','utm_medium',\n             'utm_campaign', 'utm_adcontent', 'utm_keyword',\n             'device_category', 'device_os', 'device_brand', 'device_browser',\n             'geo_country',\t'geo_city', 'hit_referer',\n             'hit_page_path', 'event_category', 'event_action', 'event_label']\ntrain_pool = Pool(X, y, cat_features=cat_feats)\n\n# обучение\nmodel = CatBoostClassifier(\n    iterations=100,\n    learning_rate=0.1,\n    depth=6,\n    verbose=50\n)\nmodel.fit(train_pool)\n"

In [ ]:
'''
model = CatBoostClassifier(
    iterations=200,
    learning_rate=0.1,
    depth=6,
    thread_count=-1,
    early_stopping_rounds=20,
    verbose=50
)

model.fit(
    train_pool,
    eval_set=val_pool,
    use_best_model=True
)
'''

0:	test: 1.0000000	best: 1.0000000 (0)	total: 6.8s	remaining: 1h 53m 12s
